# Step-1 Collecting the data 

In [ ]:
import cv2,os

# Load Haarcascade face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# function for detecting faces
# it returns cropped faces
def face_extractor(img):
    
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces == ():
        return None
    
    # cropping the faces
    for (x,y,w,h) in faces:
        crop_face = img[y:y+h, x:x+w]

    return crop_face

# Collecting data from two users
for i in range(2):
    
    user = input("enter user number:")
    cap = cv2.VideoCapture(0)
    count = 0

# Collect 100 samples from each user and store in a seperate file
    while True:
        

        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            path1= "./faces/"
            path = os.path.join(path1,user,"")
            # saving the images to a specific file
            file_name_path = path + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)
        
                

        # To put count on each image while collecting the images
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)
        
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: 
            break
        
    cap.release()
    cv2.destroyAllWindows()      
    print("Collecting Samples Complete")

# Step-2 Training the models

In [ ]:
import cv2,os
import numpy as np
from os import listdir
from os.path import isfile, join
# iterating twice to train two models
for j in range(2):
    # fetching the training data
    data_path = path = os.path.join("./faces/",str(j+1),"")
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

#creating lists for training data and labels
    Training_Data , Labels = [] , []

# Reading the samples from file
# Create numpy array for training data
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)
   
    if j==0:
        
        ms_model  = cv2.face_LBPHFaceRecognizer.create()
      # training the model 
        ms_model.train(np.asarray(Training_Data),np.asarray(Labels))
        print("MS Model trained sucessefully")
    else:
        virat_model = cv2.face_LBPHFaceRecognizer.create()
        virat_model.train(np.asarray(Training_Data), np.asarray(Labels))
        print(" Virat Model trained sucessefully")

# Step-3 Face recognition and unlock

In [ ]:
import cv2,os
import numpy as np
import pywhatkit, datetime
from time import sleep
import smtplib,getpass

def whatsapp_msg():
    # to get current time use datetime.now function
    c = datetime.datetime.now()
    #enter phone number
    phno = getpass.getpass("enter number:")
    # enter the message
    message = input("enter message:")
    # to send whatsapp message use sendwhatmsg function
    pywhatkit.sendwhatmsg(phno,message,c.hour,c.minute+2)


def send_email():
    # create session
    s = smtplib.SMTP('smtp.gmail.com', 587)
    # start TLS
    s.starttls()
    # Authentication
    sender = input("enter email: ")
    # to take input as password use getpass
    password = getpass.getpass("enter password: ")    
    # sender login
    s.login(sender,password)   
    recipient = input("enter the recipient email: ")
    # enter the message that is to be sent
    message = "hey Mahi, just recognized you."
    # sending the mail to recipient
    s.sendmail(sender,recipient,message)
    # terminate the session
    s.quit()    
    print("email sent")
    
def aws():
    # to launch instance
    os.system("aws ec2 run-instances --image-id ami-0ad704c126371a549 --instance-type t2.micro --count 1 --subnet-id subnet-e17f8a8a --security-group-ids sg-01e582a3eb9d4dac3 --key-name summertraining")
    print("launched")
    # to create volume
    os.system("aws ec2 create-volume --availability-zone ap-south-1a --size 5 --tag-specifications ResourceType=volume,Tags={Key=name,Value=vol2}")
    print("volume created")
    i_id = input("enter instance id:")
    v_id = input("enter volume id:")
    # to attach volume
    os.system("aws ec2 attach-volume --instance-id "+i_id+" --volume-id "+v_id+" --device /dev/xvdf" )
    print("attached")


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    #to crop and resize face
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()   
    image, face = face_detector(frame)   
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

    #passing the face to the models to predict    
    results = ms_model.predict(face)
    results2 = virat_model.predict(face)
        
        
    if results[1] < 500:
        confidence = int( 100 * (1 - (results[1])/400) )
    if results2[1] < 500:
        confidence1 = int( 100 * (1 - (results2[1])/400) )
        
    if confidence > 85:
        cv2.putText(image, "Hey MS", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image )
        cv2.waitKey(100)
        sleep(5)
        cv2.destroyAllWindows()
        cap.release()
        #calling functions to send email and whatsapp message
        send_email()
        whatsapp_msg()
            
        break
    elif confidence1 > 85:
        cv2.putText(image, "Hey Virat", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image )
        cv2.waitKey(100)
        sleep(5)
        cv2.destroyAllWindows()
        cap.release()
        # calling function to launch instance, create and attach volume
        aws()
        break
         
   
        
    else:
        
        cv2.putText(image, "Who are you?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image )  
        cv2.waitKey(100)
        sleep(5)
        cv2.destroyAllWindows()
        cap.release()
        break

    
        
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()     